In [14]:
import pandas as pd
import numpy as np
import time
import re
import requests
import pickle
import joblib
import lxml
import joblib

from tensorflow.keras.preprocessing import sequence
from eunjeon import Mecab
from keras.models import load_model
from selenium import webdriver
from bs4 import BeautifulSoup
from dateutil.relativedelta import relativedelta
from datetime import datetime

In [15]:
정상재무비율 = pd.read_csv('./rawdata/정상기업재무비율21년도.csv', encoding='cp949')
정상재무비율.회사명 = 정상재무비율.회사명.str.replace('\(주\)', '')
정상기사 = pd.read_csv('./rawdata/정상기업번역파일21.csv', index_col=0)
정상기사.기업 = 정상기사.기업.str.replace('\(주\)', '')

C:\Users\dbswo\AppData\Local\Temp\ipykernel_15020\3906184618.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  정상재무비율.회사명 = 정상재무비율.회사명.str.replace('\(주\)', '')
C:\Users\dbswo\AppData\Local\Temp\ipykernel_15020\3906184618.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  정상기사.기업 = 정상기사.기업.str.replace('\(주\)', '')


In [16]:
mecab = Mecab()
## Mecab으로 명사에 해당하는 단어만 추출합니다.
정상기사['뉴스기사본문전처리'] = 정상기사['뉴스기사본문전처리'].apply(lambda x: mecab.nouns(x))
## 한 글자 단어를 제거합니다.
정상기사['뉴스기사본문전처리'] = 정상기사['뉴스기사본문전처리'].apply(lambda x: [word for word in x if len(word) > 1])

In [17]:
with open('./model/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

뉴스기사본문전처리 = np.array(정상기사['뉴스기사본문전처리'])

뉴스기사본문전처리 = tokenizer.texts_to_sequences(뉴스기사본문전처리)
뉴스기사본문전처리 = sequence.pad_sequences(뉴스기사본문전처리, maxlen=100)

In [18]:
model_t = load_model("./model/03-0.150524.hdf5")

predict = model_t.predict(뉴스기사본문전처리)
predict = predict.round()

## 뉴스 데이터프레임에 예측값을 부도기사분류라는 이름으로 붙이고 groupby를 활용하여 부도기사비율을 계산합니다.
정상기사['부도기사분류'] = predict
전체기사수 = pd.DataFrame(정상기사.groupby(['기업']).count()['부도기사분류'])
전체기사수.reset_index(drop = False, inplace = True)
전체기사수.columns = ['회사명', '전체기사수']

부도기사수 = pd.DataFrame(정상기사[정상기사.부도기사분류==1].groupby(['기업']).count()['부도기사분류'])
부도기사수.reset_index(drop = False, inplace = True)
부도기사수.columns = ['회사명', '부도기사수']

부도기사비율 = pd.merge(전체기사수, 부도기사수, on='회사명')
부도기사비율['부도기사비율'] = (부도기사비율['부도기사수'] / 부도기사비율['전체기사수']) * 100
부도기사비율.drop(['전체기사수', '부도기사수'], axis=1, inplace=True)

2/2 [==============================] - 0s 6ms/step


In [19]:
사용할재무비율_최종  = pd.merge(정상재무비율, 부도기사비율, on='회사명', how='left')
사용할재무비율_최종['부도기사비율'].fillna(0, inplace=True)
사용할재무비율_최종.to_csv('정상기업데이터.csv', encoding='utf-8-sig', index=None)